<a href="https://colab.research.google.com/github/MatinAminSabouri/deepseekv3_finetuning/blob/main/QwenFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
from google.colab import files
import json
import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-yf42f9r9/unsloth_bf247369b9b24cb49d4d01da5c297161
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-yf42f9r9/unsloth_bf247369b9b24cb49d4d01da5c297161
  Resolved https://github.com/unslothai/unsloth.git to commit aa5832de9282987ae6221dfac1877d23d64cad9a


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-7B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

EOS_TOKEN = tokenizer.eos_token
FastLanguageModel.for_inference(model)

In [ ]:
# آپلود فایل JSON
uploaded = files.upload()

# پردازش JSON
def preprocess_json(json_data):
    processed_data = []

    # بررسی وجود sections
    if "sections" not in json_data or not isinstance(json_data["sections"], list):
        print("Error: 'sections' key missing or not a list.")
        return processed_data

    # گرفتن زمینه (context)
    main_context = ""
    if len(json_data["sections"]) > 0:
        main_context = json_data["sections"][0].get("content", "")

    # گرفتن FAQها
    faq_content = ""
    if len(json_data["sections"]) > 1:
        faq_content = json_data["sections"][1].get("content", "")
    else:
        print("Warning: Only one section found. Looking for FAQ content in sections[0].")
        faq_content = json_data["sections"][0].get("content", "")

    # پردازش faq_content
    if isinstance(faq_content, str):
        faq_lines = [line.strip().lstrip("- ") for line in faq_content.replace("- ", "\n").split("\n") if line.strip()]
        current_question = None
        for line in faq_lines:
            if line.endswith("؟"):
                current_question = line
            elif current_question and line:
                processed_data.append({
                    "question": current_question,
                    "answer": line,
                    "context": main_context
                })
                current_question = None
    elif isinstance(faq_content, list):
        for item in faq_content:
            if isinstance(item, dict) and item.get("section_type") == "faq" and "heading" in item and "content" in item:
                question = item["heading"].replace("سوال", "").strip().lstrip("0123456789: ")
                answer = item["content"].strip().lstrip("- ")
                if question and answer:
                    processed_data.append({
                        "question": question,
                        "answer": answer,
                        "context": main_context
                    })

    # نمونه‌های منفی
    negative_examples = [
        {"question": "پایتخت ایران کجاست؟", "answer": "من فقط می‌توانم درباره اطلاعات سامانه رهاورد پاسخ دهم.", "context": main_context},
        {"question": "تو کی هستی؟", "answer": "من فقط می‌توانم درباره اطلاعات سامانه رهاورد پاسخ دهم.", "context": main_context}
    ]
    processed_data.extend(negative_examples)

    return processed_data

# پردازش همه فایل‌های JSON
all_processed_data = []
for dataset_file in uploaded.keys():
    with open(dataset_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    all_processed_data.extend(preprocess_json(data))

# تبدیل به دیتاست
dataset = Dataset.from_list(all_processed_data)
print(f"Number of samples: {len(dataset)}")
if len(dataset) > 0:
    print("First sample:", dataset[0])
else:
    print("Error: No samples in dataset. Check JSON structure.")

In [ ]:
# پرامپت فاین‌تیونینگ (مشابه پرامپت تست)
fine_tune_prompt = """فقط چانک‌های ارائه‌شده را بررسی کن. اگر سؤالم دقیقاً با یکی از چانک‌ها مرتبط باشد (یعنی سؤالم یا کلمات کلیدی آن در چانک وجود داشته باشد)، آن چانک را بدون هیچ تغییر یا افزودن محتوای اضافی به‌عنوان پاسخ برگردان. اگر هیچ چانک مرتبطی پیدا نشد یا سؤالم بی‌ربط بود، فقط بگو: «من فقط می‌توانم درباره اطلاعات سامانه رهاورد پاسخ دهم.» تحت هیچ شرایطی محتوای جدید تولید نکن یا از دانش عمومی خودت استفاده نکن.

### چانک‌ها:
{context}

### سؤالم:
{question}

### پاسخ:
{answer}"""

def formatting_prompts_func(examples):
    texts = []
    for question, answer, context in zip(examples["question"], examples["answer"], examples["context"]):
        text = fine_tune_prompt.format(context=context, question=question, answer=answer) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# اعمال پرامپت به دیتاست
dataset = dataset.map(formatting_prompts_func, batched=True)
dataset = dataset.remove_columns(["question", "answer", "context"])
print("Prompts formatted. Sample:", dataset[0]["text"][:200] + "...")

In [ ]:
# تنظیم LoRA برای فاین‌تیونینگ
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# تنظیمات آموزش
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # برای دیتاست کوچک
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="farsi_qwen_rag_model",
    ),
)

print("Training configured.")

In [ ]:
trainer_stats = trainer.train()
print("Training completed. Stats:", trainer_stats)

In [ ]:
model.save_pretrained("farsi_qwen_rag_lora")
tokenizer.save_pretrained("farsi_qwen_rag_lora")
print("Model saved to 'farsi_qwen_rag_lora'")

In [ ]:
import shutil
from google.colab import files

shutil.make_archive("farsi_qwen_rag_model", 'zip', "farsi_qwen_rag_lora")
files.download("farsi_qwen_rag_model.zip")
print("Model downloaded. For usage: load LoRA with PEFT and Unsloth.")

In [ ]:
FastLanguageModel.for_inference(model)

# همان پرامپت سخت‌گیرانه
concat_prompt = """فقط چانک‌های ارائه‌شده را بررسی کن. اگر سؤالم دقیقاً با یکی از چانک‌ها مرتبط باشد (یعنی سؤالم یا کلمات کلیدی آن در چانک وجود داشته باشد)، آن چانک را بدون هیچ تغییر یا افزودن محتوای اضافی به‌عنوان پاسخ برگردان. اگر هیچ چانک مرتبطی پیدا نشد یا سؤالم بی‌ربط بود، فقط بگو: «من فقط می‌توانم درباره اطلاعات سامانه رهاورد پاسخ دهم.» تحت هیچ شرایطی محتوای جدید تولید نکن یا از دانش عمومی خودت استفاده نکن.

### چانک‌ها:
{}

### سؤالم:
{}

### پاسخ:
{}"""

# چانک‌های نمونه
test_chunks = """
- نوع درخواست باید براساس تقاضای درخواست‌کننده انتخاب شود. هر نوع درخواست روند و قوانین خاص خود را دارد.
- مالک باید مجوز نقل و انتقال را دریافت کند. این مجوز معمولاً طی چند روز تا یک هفته صادر می‌شود و مدت اعتبار آن بین یک تا سه ماه است.
- بله، درخواست باید روی یک پرونده موجود ثبت شود. بنابراین ابتدا لازم است پرونده تشکیل شود.
"""

# سؤالم
#test_question = "پایتخت ایران کجاست؟"  # باید پیغام خطا بده
#test_question = "نوع درخواست چگونه انتخاب می‌شود؟"  # باید چانک مرتبط رو برگردونه
test_question = "آیا قبل از ثبت درخواست، تشکیل پرونده الزامی است؟"

# فیلتر کردن چانک‌ها
def filter_chunk(question, chunks):
    if not chunks.strip():
        return None
    chunk_lines = [line.strip().lstrip("- ") for line in chunks.split("\n") if line.strip()]
    for chunk in chunk_lines:
        if any(word in chunk for word in question.split()):
            return chunk
    return None

selected_chunk = filter_chunk(test_question, test_chunks)

# ایجاد پرامپت
if selected_chunk:
    test_prompt = concat_prompt.format(test_chunks, test_question, selected_chunk)
else:
    test_prompt = concat_prompt.format(test_chunks, test_question, "من فقط می‌توانم درباره اطلاعات سامانه رهاورد پاسخ دهم.")

# تولید پاسخ
inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True, temperature=0.00001, top_p=0.9)
generated = tokenizer.batch_decode(outputs)[0]
print("پاسخ تولیدشده:", generated.split("### پاسخ:")[-1].strip())